# Three Dimensions


Advect particles with climatological mean flow in three dimensions starting from a selected depth level
(e.g. `k=10` for 95 m) and region using a near-global ocean state estimate ([OCCA](https://doi.org/10.1175/2009JPO4043.1)
which is here repeated for two years. For additional documentation e.g. see :
[1](https://JuliaClimate.github.io/MeshArrays.jl/dev/),
[2](https://JuliaClimate.github.io/IndividualDisplacements.jl/dev/),
[3](https://docs.juliadiffeq.org/latest/solvers/ode_solve.html),
[4](https://en.wikipedia.org/wiki/Displacement_(vector))

![Three dimensional simulation 1/2](https://user-images.githubusercontent.com/20276764/94491655-9cb95780-01b6-11eb-9273-173865ed6340.png)
![Three dimensional simulation 2/2](https://user-images.githubusercontent.com/20276764/94491485-595ee900-01b6-11eb-95e6-c2cacb812f46.png)

## 1. Load Software

In [1]:
using IndividualDisplacements, DataFrames, OceanStateEstimation, NetCDF
p=dirname(pathof(IndividualDisplacements))
include(joinpath(p,"../examples/example123.jl"))
include(joinpath(p,"../examples/helper_functions.jl"))
get_ll360_grid_if_needed(); get_occa_velocity_if_needed();

## 2.1 Ocean Circulation Setup

In [2]:
nam="OCCA"
bck=false

if nam=="OCCA"
   𝑃,Γ=OCCA_setup(backward_in_time=bck)
   🚄 =dxyz_dt
elseif nam=="LL90"
   𝑃,Γ=example3_setup(backward_in_time=bck)
   🚄 =dxy_dt
else
   error("unknown example (nam parameter value)")
end

dxyz_dt (generic function with 3 methods)

## 2.2 Solver And Analysis Setup

In [3]:
function ∫(prob)
   sol=solve(prob,Tsit5(),saveat=10*86400.0)
   nx,ny=𝑃.ioSize[1:2]
   sol[1,:,:]=mod.(sol[1,:,:],nx)
   sol[2,:,:]=mod.(sol[2,:,:],ny)
   return sol
end

function 🔧(sol,𝑃::NamedTuple;id=missing,𝑇=missing)
   df=postprocess_lonlat(sol,𝑃,id=id,𝑇=𝑇)

   #add year (convenience time axis for plotting)
   df.year=df.t ./86400/365

   #add depth (i.e. the 3rd, vertical, coordinate)
   k=sol[3,:,:]
   df.k=k[:] #level
   k=Int.(floor.(df.k)); w=(df.k-k);
   df.z=𝑃.RF[1 .+ k].*(1 .- w)+𝑃.RF[2 .+ k].*w #depth

   #add one isotherm depth
   θ=0.5*(𝑃.θ0+𝑃.θ1)
   d=isosurface(θ,15,𝑃.RC)
   d[findall(isnan.(d))].=0.
   df.iso=interp_to_xy(df,exchange(d));

   #add color = f(iso-z)
   c=fill(:gold,length(df.iso))
   c[findall(df.iso.<df.z)].=:violet
   df.col=c

   #to plot e.g. Pacific Ocean transports, shift longitude convention?
   df.lon[findall(df.lon .< 0.0 )] = df.lon[findall(df.lon .< 0.0 )] .+360.0
   return df
end

🔧 (generic function with 1 method)

## 2.3 Initialize Individuals

In [4]:
"""
    set_up_individuals(𝑃,Γ,∫,🚄,🔧; nf=10000, z_init=4.5,
               lon_rng=(-160.0,-150.0), lat_rng=(30.0,40.0))

Set up `Individuals` data structure with `nf` particles moving within a near-global Ocean domain.
"""
function set_up_individuals(𝑃,Γ,∫,🚄,🔧; nf=10000,
      z_init=4.5, lon_rng=(-160.0,-150.0), lat_rng=(30.0,40.0))

   lo0,lo1=lon_rng
   la0,la1=lat_rng

   lon=lo0 .+(lo1-lo0).*rand(nf)
   lat=la0 .+(la1-la0).*rand(nf)
   (xy,_)=initialize_lonlat(Γ,lon,lat)
   xy[3,:] .= z_init
   id=collect(1:size(xy,2))

   tr = DataFrame([fill(Int, 2) ; fill(Float64, 9); fill(Symbol, 1)],
   [:ID, :fid, :x, :y, :k, :z, :iso, :t, :lon, :lat, :year, :col])

   𝐼 = Individuals{Float64}(📌=xy, 🔴=tr, 🆔=id, 🚄 = 🚄, ∫ = ∫, 🔧 = 🔧, 𝑃=𝑃)

   return 𝐼
end

set_up_individuals(𝐼::Individuals; nf=10000) = set_up_individuals(𝑃,Γ,∫,🚄,🔧; nf=nf)

𝐼=set_up_individuals(𝑃,Γ,∫,🚄,🔧,nf=100)

  📌 details     = (3, 100) Float64
  🔴 details     = (0, 12) ["ID", "fid", "x", "y", "k", "z", "iso", "t", "lon", "lat", "year", "col"]
  🆔 range       = (1, 100)
  🚄 function    = dxyz_dt
  ∫  function    = ∫
  🔧 function    = 🔧
  𝑃  details     = (:θ0, :θ1, :𝑆0, :𝑆1, :u0, :u1, :v0, :v1, :w0, :w1, :𝑇, :XC, :YC, :RF, :RC, :ioSize)


## 3.1 Compute Displacements

In [5]:
𝑇=(0.0,100*86400.0)

∫!(𝐼,𝑇)

3×100 Array{Float64,2}:
  24.6674    31.5572    29.4547   …   27.257    23.4726    26.9494 
 118.186    113.804    110.597       111.488   110.692    118.671  
   4.94591    5.54066    5.53214       6.0673    5.24216    5.45032

## 3.2 Analyze Results

The recorded simulation output, 🔴, is a in the [DataFrames](https://juliadata.github.io/DataFrames.jl/latest/) tabular format, which is easily manipulated or plotted.

- either `Plots.jl`:

In [6]:
#plot_end_points(𝐼,Γ)

- or `Makie.jl`:

In [7]:
#include(joinpath(p,"../examples/recipes_Makie.jl"))
#p=PlotMakie(𝐼.🔴,100,180.);
#display(p)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*